# Data manipulation with dplyr

In [1]:
# Some initial setup
options(digits = 3, repr.matrix.max.rows = 6)

# load tidyverse, which includes dplyr and ggplot
library("tidyverse")

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.3
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


The [tidyverse](https://www.tidyverse.org) is a collection of actively-developed `R` packages that follow a certain principle of writing code.
Here, we will primarily focus on `dplyr`, which is the part of `tidyverse` which deals with data manipulation.

First, we will start by reading some data. 
There are many functions in `tidyverse` (and `R` in general) for reading various data formats into the `R` environment as a table of data (or, `data.table`).
For example, either `read_csv` (from `tidyverse`) or `read.csv` (base `R`) can be used to read a `csv` file (e.g., a file where each line represents a row in the table, and the columns are separated by a `,`).

For this tutorial, we will be using the [titanic](https://www.kaggle.com/c/titanic/data) passenger data from Kaggle.
The schema of the data are being described in the following table.

**Variable**|**Definition**|**Key**
:-----:|:-----:|:-----:
survival|Survival|0 = No, 1 = Yes
pclass|Ticket class|1 = 1st, 2 = 2nd, 3 = 3rd
sex|Sex| 
Age|Age in years| 
sibsp|# of siblings / spouses aboard the Titanic| 
parch|# of parents / children aboard the Titanic| 
ticket|Ticket number| 
fare|Passenger fare| 
cabin|Cabin number| 
embarked|Port of Embarkation|C = Cherbourg, Q = Queenstown, S = Southampton

To get started, let's first load and take a look at the data:

In [2]:
# load the csv data
# column types can be inferred automatically, but can also be specified with col_types argument
passengers <- read_csv(
    "titanic.csv",
    col_types = cols(.default = "d", Name='c', Sex='f', Ticket='c', Embarked='c', Cabin='f'))
# cleaning some data
passengers <- passengers %>% drop_na(Embarked, Age)


# Summary of each column
summary(passengers)

  PassengerId     Survived         Pclass         Name               Sex     
 Min.   :  1   Min.   :0.000   Min.   :1.00   Length:712         male  :453  
 1st Qu.:223   1st Qu.:0.000   1st Qu.:1.00   Class :character   female:259  
 Median :445   Median :0.000   Median :2.00   Mode  :character               
 Mean   :449   Mean   :0.404   Mean   :2.24                                  
 3rd Qu.:677   3rd Qu.:1.000   3rd Qu.:3.00                                  
 Max.   :891   Max.   :1.000   Max.   :3.00                                  
                                                                             
      Age           SibSp          Parch         Ticket               Fare    
 Min.   : 0.4   Min.   :0.00   Min.   :0.00   Length:712         Min.   :  0  
 1st Qu.:20.0   1st Qu.:0.00   1st Qu.:0.00   Class :character   1st Qu.:  8  
 Median :28.0   Median :0.00   Median :0.00   Mode  :character   Median : 16  
 Mean   :29.6   Mean   :0.51   Mean   :0.43                 

## Introduction to `dplyr` verbs

In the world of `dplyr`, a _verb_ is a function that

* takes a data frame as its first argument, and
* returns another data frame as a result

Any function that meets this criteria, even if it's not necessarily a function in the `dplyr` package, can be considered a _verb_.

For example, the `head()` function in `R`, if applied to a data frame, will return a first `n` rows of a data frame.

In [3]:
head(passengers, n = 3)  # Return a data frame that consists of the first three rows of titanic

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
<dbl>,<dbl>,<dbl>,<chr>,<fct>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<fct>,<chr>
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,NA,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.28,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.92,NA,S


In this sense, the base `R` function `head()` is a verb.

The **core idea** of `dplyr` is that a vast majority of data manipulation needs can be satisfied through a combination of five verbs.

verb                 | action
-------------------- | ---------
`filter(df, ...)`    | select a subset of _rows_ by some specified condition
`select(df, ...)`    | select a subset of _columns_
`mutate(df, ...)`    | create a _new column_ (often as a function of existing columns)
`arrange(df, ...)`   | reorder (sort) _rows_ according to values of specific _columns_
`summarize(df, ...)` | aggregate and reduce a vector (column) to a single value

We will explore each of these verbs (and some additional variations within each category) below.

### Selecting rows (1/5)

The `filter(df, ...)` verb is used to select a subset of _rows_ that satisfy the conditions specified in `...`. 
The conditions must be written in a form that would evaluate to either `TRUE` or `FALSE`.

For example, if we want a data frame of passengers in the 3rd clas:

In [4]:
filter(passengers, Pclass == 3)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
<dbl>,<dbl>,<dbl>,<chr>,<fct>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<fct>,<chr>
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,NA,S
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.92,NA,S
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,NA,S
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
885,0,3,"Sutehall, Mr. Henry Jr",male,25,0,0,SOTON/OQ 392076,7.05,NA,S
886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39,0,5,382652,29.12,NA,Q
891,0,3,"Dooley, Mr. Patrick",male,32,0,0,370376,7.75,NA,Q


Multiple conditions can also be specified.

For example, if we want a data frame of all passengers in the 3rd class that are older than 50 years old:

In [5]:
filter(passengers, Pclass == 3, Age > 50)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
<dbl>,<dbl>,<dbl>,<chr>,<fct>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<fct>,<chr>
95,0,3,"Coxon, Mr. Daniel",male,59.0,0,0,364500,7.25,NA,S
117,0,3,"Connors, Mr. Patrick",male,70.5,0,0,370369,7.75,NA,Q
153,0,3,"Meo, Mr. Alfonzo",male,55.5,0,0,A.5. 11206,8.05,NA,S
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
484,1,3,"Turkula, Mrs. (Hedwig)",female,63,0,0,4134,9.59,NA,S
632,0,3,"Lundahl, Mr. Johan Svensson",male,51,0,0,347743,7.05,NA,S
852,0,3,"Svensson, Mr. Johan",male,74,0,0,347060,7.78,NA,S


By default, `filter()` will combine multiple conditions as `AND` operations.
In the example above, we are returned passengers where `Pclass == 3` _**AND**_ `Age < 18`.

We can specify an OR condition by using the `|` operator.

For example, if we want passengers in the 3rd class who are either younger than 18 years old or older than 65 years old:

In [6]:
filter(passengers, Pclass == 3, Age > 50 | Age < 18)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
<dbl>,<dbl>,<dbl>,<chr>,<fct>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<fct>,<chr>
8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,21.07,NA,S
11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4,1,1,PP 9549,16.70,G6,S
15,0,3,"Vestrom, Miss. Hulda Amanda Adolfina",female,14,0,0,350406,7.85,NA,S
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
853,0,3,"Boulos, Miss. Nourelain",female,9,1,1,2678,15.25,NA,C
870,1,3,"Johnson, Master. Harold Theodor",male,4,1,1,347742,11.13,NA,S
876,1,3,"Najib, Miss. Adele Kiamie ""Jane""",female,15,0,0,2667,7.22,NA,C


Use the `%in%` operator to filter to values that match a collection of values. 

For example, suppose we want to look at passengers embarked on Queenstown (`Q`) and Southampton (`S`)

In [7]:
filter(passengers, 
       Embarked %in% c("Q", "S"))

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
<dbl>,<dbl>,<dbl>,<chr>,<fct>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<fct>,<chr>
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,NA,S
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.92,NA,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.10,C123,S
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
887,0,2,"Montvila, Rev. Juozas",male,27,0,0,211536,13.00,NA,S
888,1,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,30.00,B42,S
891,0,3,"Dooley, Mr. Patrick",male,32,0,0,370376,7.75,NA,Q


Finally, use `!` to negate any condition. 

For example, if we wanted to find all passengers who did NOT embark on Queenstown (`Q`) and Southampton (`S`)

In [8]:
filter(passengers, 
       !(Embarked %in% c("Q", "S")))

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
<dbl>,<dbl>,<dbl>,<chr>,<fct>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<fct>,<chr>
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.3,C85,C
10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,30.1,NA,C
31,0,1,"Uruchurtu, Don. Manuel E",male,40,0,0,PC 17601,27.7,NA,C
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
876,1,3,"Najib, Miss. Adele Kiamie ""Jane""",female,15,0,0,2667,7.22,NA,C
880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56,0,1,11767,83.16,C50,C
890,1,1,"Behr, Mr. Karl Howell",male,26,0,0,111369,30.00,C148,C


### Exercise: `filter()`
Find all pasengers with 2 siblings and 2 parents aboard.

In [9]:
# YOUR CODE HERE
filter(passengers, SibSp == 2, Parch == 2)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
<dbl>,<dbl>,<dbl>,<chr>,<fct>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<fct>,<chr>
148,0,3,"Ford, Miss. Robina Maggie ""Ruby""",female,9,2,2,W./C. 6608,34.4,NA,S
312,1,1,"Ryerson, Miss. Emily Borie",female,18,2,2,PC 17608,262.4,B57 B59 B63 B66,C
437,0,3,"Ford, Miss. Doolina Margaret ""Daisy""",female,21,2,2,W./C. 6608,34.4,NA,S
743,1,1,"Ryerson, Miss. Susan Parker ""Suzette""",female,21,2,2,PC 17608,262.4,B57 B59 B63 B66,C


### Selecting columns (2/5)

Use `select(df, ...)` to either specify which columns to select,

In [10]:
select(passengers, Survived, Pclass, Sex, Age)

Survived,Pclass,Sex,Age
<dbl>,<dbl>,<fct>,<dbl>
0,3,male,22
1,1,female,38
1,3,female,26
⋮,⋮,⋮,⋮
1,1,female,19
1,1,male,26
0,3,male,32


or to specify which columns to exclude, using `-`.

In [11]:
select(passengers, -Name, -SibSp, -Parch)

PassengerId,Survived,Pclass,Sex,Age,Ticket,Fare,Cabin,Embarked
<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<chr>,<dbl>,<fct>,<chr>
1,0,3,male,22,A/5 21171,7.25,NA,S
2,1,1,female,38,PC 17599,71.28,C85,C
3,1,3,female,26,STON/O2. 3101282,7.92,NA,S
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
888,1,1,female,19,112053,30.00,B42,S
890,1,1,male,26,111369,30.00,C148,C
891,0,3,male,32,370376,7.75,NA,Q


`dplyr` also provides some useful helper functions to `select()` columns that match specific criteria.

* `starts_with(x)`: match column names that start with `x`
* `ends_with(x)`: match column names that end with `x`
* `contains(x)`: match column names that contain `x`
* `matches(x)`: match column names that match (the regular expression) `x`

where `x` is a string (in either single- or double-quotes).

For example, if we want all the columns that start with letter `s`:

In [12]:
select(passengers, starts_with('S'))

Survived,Sex,SibSp
<dbl>,<fct>,<dbl>
0,male,1
1,female,1
1,female,0
⋮,⋮,⋮
1,female,0
1,male,0
0,male,0


You can see the documentation for `select()` for details.
In general, for any `R` function, you can pull-up the documentation (if one exists) by running `?` followed by the function name.
For example, to see the documentation for `select()` as provided in the `dplyr` package, run:

In [13]:
?dplyr::select

select {dplyr},R Documentation
.data,"A tbl. All main verbs are S3 generics and provide methods for tbl_df(), dtplyr::tbl_dt() and dbplyr::tbl_dbi()."
...,"One or more unquoted expressions separated by commas. You can treat variable names like they are positions, so you can use expressions like x:y to select ranges of variables. Positive values select variables; negative values drop variables. If the first expression is negative, select() will automatically start with all variables. Use named arguments, e.g. new_name = old_name, to rename selected variables. The arguments in ... are automatically quoted and evaluated in a context where column names represent column positions. They also support unquoting and splicing. See vignette(""programming"") for an introduction to these concepts. See select helpers for more details and examples about tidyselect helpers such as starts_with(), everything(), ..."


### Create new columns (3/5)

Use `mutate(df, ...)` to create new columns, usually as a function of existing columns.

Suppose we wish to create a column called `no_sib`, indicating whether the passenger has no sibling aboard.
We would write,

In [14]:
mutate(passengers, no_sib = (SibSp == 0))

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,no_sib
<dbl>,<dbl>,<dbl>,<chr>,<fct>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<fct>,<chr>,<lgl>
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,NA,S,FALSE
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.28,C85,C,FALSE
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.92,NA,S,TRUE
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
888,1,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,30.00,B42,S,TRUE
890,1,1,"Behr, Mr. Karl Howell",male,26,0,0,111369,30.00,C148,C,TRUE
891,0,3,"Dooley, Mr. Patrick",male,32,0,0,370376,7.75,NA,Q,TRUE


We note that within a single `mutate()` function, you can refer to the new columns you've created in the previous argument.


### Exercise: `mutate()`
Create a column `num_family` representing the number of family members on board for each passenger (the sum of sibling number and parent number)

In [15]:
# YOUR CODE HERE
mutate(passengers, num_family = SibSp + Parch)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,num_family
<dbl>,<dbl>,<dbl>,<chr>,<fct>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<fct>,<chr>,<dbl>
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,NA,S,1
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.28,C85,C,1
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.92,NA,S,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
888,1,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,30.00,B42,S,0
890,1,1,"Behr, Mr. Karl Howell",male,26,0,0,111369,30.00,C148,C,0
891,0,3,"Dooley, Mr. Patrick",male,32,0,0,370376,7.75,NA,Q,0


### Sorting (4/5)

Use `arrange(df, ...)` to reorder the rows of a data frame by the value of specified columns.
Multiple conditions are arranged from left to right.

In [16]:
arrange(passengers, Pclass, Name)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
<dbl>,<dbl>,<dbl>,<chr>,<fct>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<fct>,<chr>
731,1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,211,B5,S
306,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,152,C22 C26,S
298,0,1,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,152,C22 C26,S
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
204,0,3,"Youseff, Mr. Gerious",male,45.5,0,0,2628,7.22,NA,C
112,0,3,"Zabour, Miss. Hileni",female,14.5,1,0,2665,14.45,NA,C
423,0,3,"Zimmerman, Mr. Leo",male,29.0,0,0,315082,7.88,NA,S


Use `desc()` around columns that you want to sort in `desc`ending order.

In [17]:
arrange(passengers, Pclass, desc(Name))

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
<dbl>,<dbl>,<dbl>,<chr>,<fct>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<fct>,<chr>
326,1,1,"Young, Miss. Marie Grice",female,36,0,0,PC 17760,135.6,C32,C
556,0,1,"Wright, Mr. George",male,62,0,0,113807,26.6,NA,S
156,0,1,"Williams, Mr. Charles Duane",male,51,0,1,PC 17597,61.4,NA,C
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
280,1,3,"Abbott, Mrs. Stanton (Rosa Hunt)",female,35,1,1,C.A. 2673,20.25,NA,S
747,0,3,"Abbott, Mr. Rossmore Edward",male,16,1,1,C.A. 2673,20.25,NA,S
846,0,3,"Abbing, Mr. Anthony",male,42,0,0,C.A. 5547,7.55,NA,S


### Aggregating (5/5)

Use `summarize(df, ...)` to aggregate multiple rows into a single row. Unlike `mutate()`, function that are used in `summarize()` must return a single value (i.e., "aggregate" the provided vector)

For example, to find the min, mean, and max age of all passengers,

In [18]:
summarize(passengers,
          min_height = min(Age),
          avg_height = mean(Age),
          max_height = max(Age))

min_height,avg_height,max_height
<dbl>,<dbl>,<dbl>
0.42,29.6,80


`dplyr` also provides a special function `n()` which will evaluate to the number of rows within a `dplyr` verb.

For example, to count how many passengers (rows) there are in our dataset in total, 

In [19]:
summarize(passengers, N = n())

N
<int>
712


### Exercise: `summarize()`
Calculate the mean, median and standard deviation of `Fare`.
(hint: using `median` and `sd` function for median and standard deviation respectively) 

In [20]:
# YOUR CODE HERE
summarize(passengers,
          mean = mean(Fare),
          median = median(Fare),
          sd = sd(Fare))

mean,median,sd
<dbl>,<dbl>,<dbl>
34.6,15.6,52.9


### Grouping (Split-apply-combine)

Now that we've covered the five core verbs, we should be able to manipulate data to our heart's desires, yes?

Then, how about:

* The number of passengers for each `Sex`?
* Survival rates in each `Pclass`?
* Number of each `Pclass` for passengers embarked on each port?

As an example, let's just consider the number of passengers for each `Sex`

A natural, but _**tedious**_ way to compute this would look something like this:

In [21]:
# NOTE: Code in this cell is intended to be an example of a BAD way to compute this.
#       This is purely for illustrative purposes, and should NEVER EVER be re-used, in any context.
male_passengers <- filter(passengers, Sex == "male")
female_passengers <- filter(passengers, Sex == "female")

N_males <- summarize(male_passengers, N = n())
N_females <- summarize(female_passengers, N = n())

c(males = N_males$N, females = N_females$N)

males females 
    453     259

This style of code can easily get out of hand, and would be a nightmare to maintain! 
(e.g., what happens if we get a new dataset where we want to count the number of rows for 8 different categories rather than just 2 categories like `male` and `female`?)

As horrible as the above code is, it is useful in highlighting a common pattern that emerges when manipulating data:

1. **Split**: The data are split into smaller pieces of data, according to one (or more) column. 
   In this case, we've split the data by the `Sex` column.
1. **Apply**: Some operation is applied to each of the smaller pieces.
   In this case, we've simply counted the number of rows of each piece using `summarize()` and `n()`.
1. **Combine**: The results of the previous **apply** are combined to some final data structure.
   In the above case, for simplicity, we've combinded the result as a vector; but in practice we usually want to keep everything in the form of a data frame.

This pattern in data manipulation is so common, that there is a `dplyr` verb for it. 
This is the `group_by` verb.

On it's own, `group_by` makes no visible changes to a data frame, other than marking the data frame as being "grouped".
The difference is only made apparent when we apply some other verb to a grouped data frame.

Note that none of the `dplyr` verbs make any changes to the original data frame! This is very intentional. 
So, for now, we need to save the "grouped" data as a new variable for our changes to have effect (but we'll see a more convenient approach to this later).

In [22]:
passengers_by_gender <- group_by(passengers, Sex)

# Note that the two data frames, on the surface, seem identical.
passengers
passengers_by_gender

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
<dbl>,<dbl>,<dbl>,<chr>,<fct>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<fct>,<chr>
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,NA,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.28,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.92,NA,S
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
888,1,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,30.00,B42,S
890,1,1,"Behr, Mr. Karl Howell",male,26,0,0,111369,30.00,C148,C
891,0,3,"Dooley, Mr. Patrick",male,32,0,0,370376,7.75,NA,Q


PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
<dbl>,<dbl>,<dbl>,<chr>,<fct>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<fct>,<chr>
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,NA,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.28,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.92,NA,S
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
888,1,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,30.00,B42,S
890,1,1,"Behr, Mr. Karl Howell",male,26,0,0,111369,30.00,C148,C
891,0,3,"Dooley, Mr. Patrick",male,32,0,0,370376,7.75,NA,Q


In [23]:
# But we can see a difference when applying, for example, a summarize
summarize(passengers, N = n())

N
<int>
712


In [24]:
summarize(passengers_by_gender, N = n())

Sex,N
<fct>,<int>
male,453
female,259


As shown in the simple example above, when a `dplyr` verb is applied to a "grouped" data frame,
`dplyr` internally **splits**-**applies**-and **combines** the data, finally returning results for
_each of the unique values that are found in the columns by which the data frame is grouped_.

This can be a lot to process, if it's the first time you've seen this. 
But once you get used to it (via trying a bunch of manipulation tasks and seeing some more examples), you'll find it extremely convenient and powerful.

Let's try answering the other questions we started this section with.

* Survival rates in each `Pclass`?

In [25]:
passengers_by_class <- group_by(passengers, Pclass)

summarize(passengers_by_class, 
          survival_rate = mean(Survived))

Pclass,survival_rate
<dbl>,<dbl>
1,0.652
2,0.480
3,0.239


* Number of each `Pclass` for passengers embarked on each port?

In [26]:
passengers_by_port <- group_by(passengers, Embarked, Pclass)

summarize(passengers_by_port, 
          N = n())

Embarked,Pclass,N
<chr>,<dbl>,<int>
C,1,74
C,2,15
C,3,41
⋮,⋮,⋮
S,1,108
S,2,156
S,3,290


### Exercise: `group_by()`

Calculate the mean fare and survival rate for male and female passengers respectively

In [27]:
# YOUR CODE HERE
gender_group <- group_by(passengers, Sex)
summarize(gender_group, 
          mean_fare = mean(Fare),
          survival_rate = mean(Survived)
         )

Sex,mean_fare,survival_rate
<fct>,<dbl>,<dbl>
male,27.3,0.205
female,47.3,0.753


### Multiple (chained) operations

As we've breifly seen above, we would often like to apply multiple operations (verbs) to a data frame.
However, by design, verbs do not save intermediate changes to the original data frame, so for each operation we would have to assign the result to a new data frame.

Even for a reasonable number of operations, this can get quite messy (i.e., we'd end up with so many names and data frames that we only use as intermediate steps).

Consider the following query:

* For each port, what is the proportion of each class?

We can think of finding the answer in multiple steps:

1. group by `Embarked` and `Pclass`
1. find the number of passengers for each of the groups in the previously grouped data frame
1. with the computed number of passengers for each class-port pair, re-group by only `Embarked`
1. create a new column which computes the proportion of counts for each row, over the sum of rows

Using the current method of saving all intermediate results, the implementation would look something like this:

In [28]:
# NOTE: Code in this cell is intended to be an example of a BAD implementation.
#       While this implementation is acceptable, it is intended for illustrative purposes, 
#       and is best avoided.  
passengers_by_port_and_class <- group_by(passengers, Embarked, Pclass)
counts_by_port_and_class <- summarize(passengers_by_port_and_class, N = n())
regroup_by_port <- group_by(counts_by_port_and_class, Embarked)
mutate(regroup_by_port, prop = N / sum(N))

Embarked,Pclass,N,prop
<chr>,<dbl>,<int>,<dbl>
C,1,74,0.569
C,2,15,0.115
C,3,41,0.315
⋮,⋮,⋮,⋮
S,1,108,0.195
S,2,156,0.282
S,3,290,0.523


The above code is bad for a multiple reasons. Among others, it's

* creating a lot of unnecessary intermediate results that will not be used again
* difficult to read, if you don't already know what the end goal is (e.g., you eyes have to wander left-to-right-to-left a few times to see what's going on)

A sophisticated, yet quite simple, solution to this problem is the introduction of `%>%`, also called the "pipe operator".

`%>%` is a _binary operator_ (much like `+` or `-`) which, in words, takes the result of the left-hand side, and uses it as the first argument on the right hand side. 
This may be confusing at first, but might make more sense in the context of `dplyr` _verbs_. Recall, a _verb_ in `dplyr` is any function that _returns a data frame_ (LHS) and _takes a data frame as its first argument_ (RHS).

If further notation is helpful, one could also write the `%>%` as
```
f(x) %>% g(y) = g(f(x), y)
```
which, depending on your background, could be clarifying or more confusing.

What this means from a practical standpoint, however, is that we no longer need to _save_ intermediate results just to use them in the next verb. 
Instead, we can use `%>%` to send results from a verb down a "pipe" to the next verb.
Consider our previous example, which involved four verbs, with three intermediate steps.
Using `%>%`, the same result can now be achieved in a (conceptually) single line:

In [29]:
passengers %>% 
    group_by(Embarked, Pclass) %>%
    summarize(N = n()) %>%
    group_by(Embarked) %>%
    mutate(prop = N / sum(N))

Embarked,Pclass,N,prop
<chr>,<dbl>,<int>,<dbl>
C,1,74,0.569
C,2,15,0.115
C,3,41,0.315
⋮,⋮,⋮,⋮
S,1,108,0.195
S,2,156,0.282
S,3,290,0.523


Note the intentional style of (1) starting from the data frame (instead of a verb that explicitly includes the data frame) and (2) keeping each verb on its own line. 
This not only makes it easier to read, but also easier to maintain and modify.

In [30]:
head(passengers, 1)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
<dbl>,<dbl>,<dbl>,<chr>,<fct>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<fct>,<chr>
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,NA,S


In [31]:
passengers %>%
  mutate(age_group = 10 * floor(Age / 10)) %>%
  group_by(Sex, age_group) %>%
  summarize(num_pass = n(), survival_rate = mean(Survived))

Sex,age_group,num_pass,survival_rate
<fct>,<dbl>,<int>,<dbl>
male,0,32,0.594
male,10,57,0.123
male,20,148,0.169
⋮,⋮,⋮,⋮
female,40,32,0.688
female,50,18,0.889
female,60,3,1.000


### Exercise: Introduction to `dplyr`
1. Create a column `age_group` that groups each passenger's age down to the 10s.
1. Calculate the number of passengers and survival rate for each (`Sex`, `age_group`) pair

What patterns do you observe?

(hint1: function `floor` takes in x and returns the largest integers not greater than the corresponding elements of x, e.g., `floor(1.9)` will return `1`)

(hint2: you can run `options(repr.matrix.max.rows = 20)` to display more rows at a time)


In [32]:
options(repr.matrix.max.rows = 20)
# YOUR CODE HERE

passengers  %>% 
    mutate(age_group = floor(Age / 10) * 10) %>%
    group_by(Sex, age_group) %>%
    summarize(
        N = n(),
        survival_rate = mean(Survived)
    )

Sex,age_group,N,survival_rate
<fct>,<dbl>,<int>,<dbl>
male,0,32,0.594
male,10,57,0.123
male,20,148,0.169
male,30,107,0.215
male,40,57,0.211
male,50,30,0.133
male,60,15,0.133
male,70,6,0.000
male,80,1,1.000


## Ending notes

There are many, MANY more verbs that we simply did not have the time to cover here, but are immensely useful. 
Some examples are:

* `rename(df, ...)`: rename columns
* `slice(df, ...)`: select rows of a data frame by index, instead of some condition
* `top_n(df, N, col)`: retrieve the top N rows for values in some specified column

You are highly recommended to explore more. One great resource for learning about `tidyverse` and using it to work with data is Hadley Wickham's online book: [R for Data Science](https://r4ds.had.co.nz/). 

Hadley Wickham is also the original author for many of the packages in `tidyverse`. In fact, in the "early days" (circa 2016), before the word `tidyverse` was created, the collection of Hadley Wickham's `R` packages were unofficially referred to as the `hadleyverse`, until [Hadley announced tidyverse and explicitly asked people to stop calling it the hadleyverse](https://twitter.com/hadleywickham/status/774008060549312512?lang=en).